In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import uniform, randint

EXPLORACIÓN Y TRANSFORMACIONES DE DIAMONDS_TRAIN

In [18]:
diamonds_train = pd.read_csv("data/diamonds_train.csv")
diamonds_test = pd.read_csv("data/diamonds_test.csv")
diamonds_train = diamonds_train.drop(["index_id", "city_id", "city_id.1", "index_id.1", "cut_id", "color_id", "color_id.1", "clarity_id", "cut_id.1", "index_id.2", "clarity_id.1"], axis = 1)

In [19]:
diamonds_train = diamonds_train[(diamonds_train != 0).all(axis=1)]
diamonds_train

,price,carat,city,color,cut,depth,table,x,y,z,clarity
0,4268,1.21,Dubai,J,Premium,62.4,58.0,6.83,6.79,4.25,VS2
1,505,0.32,Kimberly,H,Very Good,63.0,57.0,4.35,4.38,2.75,VS2
2,2686,0.71,Las Vegas,G,Fair,65.5,55.0,5.62,5.53,3.65,VS1
3,738,0.41,Kimberly,D,Good,63.8,56.0,4.68,4.72,3.00,SI1
4,4882,1.02,Dubai,G,Ideal,60.5,59.0,6.55,6.51,3.95,SI1
...,...,...,...,...,...,...,...,...,...,...,...
40450,10070,1.34,Antwerp,G,Ideal,62.7,57.0,7.10,7.04,4.43,VS1
40451,12615,2.02,Madrid,F,Good,57.1,60.0,8.31,8.25,4.73,SI2
40452,5457,1.01,Kimberly,H,Ideal,62.7,56.0,6.37,6.42,4.01,SI1
40453,456,0.33,Kimberly,J,Ideal,61.9,54.3,4.45,4.47,2.76,VS1


In [20]:
diamonds_train = diamonds_train.drop_duplicates()

In [21]:
diamonds_test = diamonds_test[(diamonds_test != 0).all(axis=1)]

diamonds_test=diamonds_test.drop_duplicates()

In [22]:
diamonds_train["cut_encoded"] = 0 

for i, r in diamonds_train.iterrows():
    if r["cut"] == "Fair":
        diamonds_train.at[i, "cut_encoded"] = 1
    elif r["cut"] == "Good":
        diamonds_train.at[i, "cut_encoded"] = 2
    elif r["cut"] == "Very Good":
        diamonds_train.at[i, "cut_encoded"] = 3
    elif r["cut"] == "Premium":
        diamonds_train.at[i, "cut_encoded"] = 4
    elif r["cut"] == "Ideal":
        diamonds_train.at[i, "cut_encoded"] = 5
diamonds_train

/var/folders/wk/24hqhn4d5jngl5h7k4s5bdk40000gp/T/ipykernel_31656/2550936690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diamonds_train["cut_encoded"] = 0


,price,carat,city,color,cut,depth,table,x,y,z,clarity,cut_encoded
0,4268,1.21,Dubai,J,Premium,62.4,58.0,6.83,6.79,4.25,VS2,4
1,505,0.32,Kimberly,H,Very Good,63.0,57.0,4.35,4.38,2.75,VS2,3
2,2686,0.71,Las Vegas,G,Fair,65.5,55.0,5.62,5.53,3.65,VS1,1
3,738,0.41,Kimberly,D,Good,63.8,56.0,4.68,4.72,3.00,SI1,2
4,4882,1.02,Dubai,G,Ideal,60.5,59.0,6.55,6.51,3.95,SI1,5
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,10070,1.34,Antwerp,G,Ideal,62.7,57.0,7.10,7.04,4.43,VS1,5
40451,12615,2.02,Madrid,F,Good,57.1,60.0,8.31,8.25,4.73,SI2,2
40452,5457,1.01,Kimberly,H,Ideal,62.7,56.0,6.37,6.42,4.01,SI1,5
40453,456,0.33,Kimberly,J,Ideal,61.9,54.3,4.45,4.47,2.76,VS1,5


In [23]:
diamonds_train["clarity_encoded"] = 0 

for i, r in diamonds_train.iterrows():
    if r["clarity"] == "I1":
        diamonds_train.at[i, "clarity_encoded"] = 1
    elif r["clarity"] == "SI2":
        diamonds_train.at[i, "clarity_encoded"] = 2
    elif r["clarity"] == "SI1":
        diamonds_train.at[i, "clarity_encoded"] = 3
    elif r["clarity"] == "VS2":
        diamonds_train.at[i, "clarity_encoded"] = 4
    elif r["clarity"] == "VS1":
        diamonds_train.at[i, "clarity_encoded"] = 5
    elif r["clarity"] == "VVS2":
        diamonds_train.at[i, "clarity_encoded"] = 6
    elif r["clarity"] == "VVS1":
        diamonds_train.at[i, "clarity_encoded"] = 7
    elif r["clarity"] == "IF":
        diamonds_train.at[i, "clarity_encoded"] = 8
diamonds_train

/var/folders/wk/24hqhn4d5jngl5h7k4s5bdk40000gp/T/ipykernel_31656/4073681236.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diamonds_train["clarity_encoded"] = 0


,price,carat,city,color,cut,depth,table,x,y,z,clarity,cut_encoded,clarity_encoded
0,4268,1.21,Dubai,J,Premium,62.4,58.0,6.83,6.79,4.25,VS2,4,4
1,505,0.32,Kimberly,H,Very Good,63.0,57.0,4.35,4.38,2.75,VS2,3,4
2,2686,0.71,Las Vegas,G,Fair,65.5,55.0,5.62,5.53,3.65,VS1,1,5
3,738,0.41,Kimberly,D,Good,63.8,56.0,4.68,4.72,3.00,SI1,2,3
4,4882,1.02,Dubai,G,Ideal,60.5,59.0,6.55,6.51,3.95,SI1,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,10070,1.34,Antwerp,G,Ideal,62.7,57.0,7.10,7.04,4.43,VS1,5,5
40451,12615,2.02,Madrid,F,Good,57.1,60.0,8.31,8.25,4.73,SI2,2,2
40452,5457,1.01,Kimberly,H,Ideal,62.7,56.0,6.37,6.42,4.01,SI1,5,3
40453,456,0.33,Kimberly,J,Ideal,61.9,54.3,4.45,4.47,2.76,VS1,5,5


In [24]:
diamonds_train["color_encoded"] = 0 

for i, r in diamonds_train.iterrows():
    if r["color"] == "J":
        diamonds_train.at[i, "color_encoded"] = 1
    elif r["color"] == "I":
        diamonds_train.at[i, "color_encoded"] = 2
    elif r["color"] == "H":
        diamonds_train.at[i, "color_encoded"] = 3
    elif r["color"] == "G":
        diamonds_train.at[i, "color_encoded"] = 4
    elif r["color"] == "F":
        diamonds_train.at[i, "color_encoded"] = 5
    elif r["color"] == "E":
        diamonds_train.at[i, "color_encoded"] = 6
    elif r["color"] == "D":
        diamonds_train.at[i, "color_encoded"] = 7
diamonds_train

/var/folders/wk/24hqhn4d5jngl5h7k4s5bdk40000gp/T/ipykernel_31656/2001936109.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diamonds_train["color_encoded"] = 0


,price,carat,city,color,cut,depth,table,x,y,z,clarity,cut_encoded,clarity_encoded,color_encoded
0,4268,1.21,Dubai,J,Premium,62.4,58.0,6.83,6.79,4.25,VS2,4,4,1
1,505,0.32,Kimberly,H,Very Good,63.0,57.0,4.35,4.38,2.75,VS2,3,4,3
2,2686,0.71,Las Vegas,G,Fair,65.5,55.0,5.62,5.53,3.65,VS1,1,5,4
3,738,0.41,Kimberly,D,Good,63.8,56.0,4.68,4.72,3.00,SI1,2,3,7
4,4882,1.02,Dubai,G,Ideal,60.5,59.0,6.55,6.51,3.95,SI1,5,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,10070,1.34,Antwerp,G,Ideal,62.7,57.0,7.10,7.04,4.43,VS1,5,5,4
40451,12615,2.02,Madrid,F,Good,57.1,60.0,8.31,8.25,4.73,SI2,2,2,5
40452,5457,1.01,Kimberly,H,Ideal,62.7,56.0,6.37,6.42,4.01,SI1,5,3,3
40453,456,0.33,Kimberly,J,Ideal,61.9,54.3,4.45,4.47,2.76,VS1,5,5,1


In [27]:
diamonds_train = diamonds_train.drop(["city", "color", "cut","clarity"], axis=1)

KeyError: "['city', 'color', 'cut', 'clarity'] not found in axis"

In [28]:
diamonds_train

,price,carat,depth,table,x,y,z,cut_encoded,clarity_encoded,color_encoded
0,4268,1.21,62.4,58.0,6.83,6.79,4.25,4,4,1
1,505,0.32,63.0,57.0,4.35,4.38,2.75,3,4,3
2,2686,0.71,65.5,55.0,5.62,5.53,3.65,1,5,4
3,738,0.41,63.8,56.0,4.68,4.72,3.00,2,3,7
4,4882,1.02,60.5,59.0,6.55,6.51,3.95,5,3,4
...,...,...,...,...,...,...,...,...,...,...
40450,10070,1.34,62.7,57.0,7.10,7.04,4.43,5,5,4
40451,12615,2.02,57.1,60.0,8.31,8.25,4.73,2,2,5
40452,5457,1.01,62.7,56.0,6.37,6.42,4.01,5,3,3
40453,456,0.33,61.9,54.3,4.45,4.47,2.76,5,5,1


EXPLORACIÓN Y TRANSFORMACIONES DE DIAMONDS_TEST

In [29]:
diamonds_test["cut_encoded"] = 0 

for i, r in diamonds_test.iterrows():
    if r["cut"] == "Fair":
        diamonds_test.at[i, "cut_encoded"] = 1
    elif r["cut"] == "Good":
        diamonds_test.at[i, "cut_encoded"] = 2
    elif r["cut"] == "Very Good":
        diamonds_test.at[i, "cut_encoded"] = 3
    elif r["cut"] == "Premium":
        diamonds_test.at[i, "cut_encoded"] = 4
    elif r["cut"] == "Ideal":
        diamonds_test.at[i, "cut_encoded"] = 5
diamonds_test

,id,carat,cut,color,clarity,depth,table,x,y,z,city,cut_encoded
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat,5
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly,4
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly,3
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam,3
5,5,1.01,Premium,F,VS2,61.3,61.0,6.38,6.34,3.90,Paris,4
...,...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam,5
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City,5
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv,5
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat,3


In [30]:
diamonds_test["clarity_encoded"] = 0 

for i, r in diamonds_test.iterrows():
    if r["clarity"] == "I1":
        diamonds_test.at[i, "clarity_encoded"] = 1
    elif r["clarity"] == "SI2":
        diamonds_test.at[i, "clarity_encoded"] = 2
    elif r["clarity"] == "SI1":
        diamonds_test.at[i, "clarity_encoded"] = 3
    elif r["clarity"] == "VS2":
        diamonds_test.at[i, "clarity_encoded"] = 4
    elif r["clarity"] == "VS1":
        diamonds_test.at[i, "clarity_encoded"] = 5
    elif r["clarity"] == "VVS2":
        diamonds_test.at[i, "clarity_encoded"] = 6
    elif r["clarity"] == "VVS1":
        diamonds_test.at[i, "clarity_encoded"] = 7
    elif r["clarity"] == "IF":
        diamonds_test.at[i, "clarity_encoded"] = 8
diamonds_test

,id,carat,cut,color,clarity,depth,table,x,y,z,city,cut_encoded,clarity_encoded
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat,5,5
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly,4,3
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly,3,3
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam,3,5
5,5,1.01,Premium,F,VS2,61.3,61.0,6.38,6.34,3.90,Paris,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam,5,3
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City,5,4
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv,5,5
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat,3,2


In [31]:
diamonds_test["color_encoded"] = 0 

for i, r in diamonds_test.iterrows():
    if r["color"] == "J":
        diamonds_test.at[i, "color_encoded"] = 1
    elif r["color"] == "I":
        diamonds_test.at[i, "color_encoded"] = 2
    elif r["color"] == "H":
        diamonds_test.at[i, "color_encoded"] = 3
    elif r["color"] == "G":
        diamonds_test.at[i, "color_encoded"] = 4
    elif r["color"] == "F":
        diamonds_test.at[i, "color_encoded"] = 5
    elif r["color"] == "E":
        diamonds_test.at[i, "color_encoded"] = 6
    elif r["color"] == "D":
        diamonds_test.at[i, "color_encoded"] = 7
diamonds_test

,id,carat,cut,color,clarity,depth,table,x,y,z,city,cut_encoded,clarity_encoded,color_encoded
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat,5,5,1
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly,4,3,3
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly,3,3,5
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam,3,5,5
5,5,1.01,Premium,F,VS2,61.3,61.0,6.38,6.34,3.90,Paris,4,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam,5,3,6
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City,5,4,2
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv,5,5,5
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat,3,2,5


In [32]:
diamonds_test = diamonds_test.drop(["city", "color", "cut","clarity", "id"], axis=1)

In [33]:
diamonds_train["volume"] = diamonds_train["x"] * diamonds_train["y"] * diamonds_train["z"] * (4/3) * np.pi
diamonds_test["volume"] = diamonds_test["x"] * diamonds_test["y"] * diamonds_test["z"] * (4/3) * np.pi

In [34]:
diamonds_train["depth_normal"] = 0
for i, r in diamonds_train.iterrows():
    if 55 < r["depth"] < 68:
        diamonds_train.at[i, "depth_normal"] = 1
    else:
        diamonds_train.at[i, "depth_normal"] = 0
diamonds_train

,price,carat,depth,table,x,y,z,cut_encoded,clarity_encoded,color_encoded,volume,depth_normal
0,4268,1.21,62.4,58.0,6.83,6.79,4.25,4,4,1,825.596831,1
1,505,0.32,63.0,57.0,4.35,4.38,2.75,3,4,3,219.474804,1
2,2686,0.71,65.5,55.0,5.62,5.53,3.65,1,5,4,475.163334,1
3,738,0.41,63.8,56.0,4.68,4.72,3.00,2,3,7,277.586100,1
4,4882,1.02,60.5,59.0,6.55,6.51,3.95,5,3,4,705.517829,1
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,10070,1.34,62.7,57.0,7.10,7.04,4.43,5,5,4,927.520129,1
40451,12615,2.02,57.1,60.0,8.31,8.25,4.73,2,2,5,1358.328217,1
40452,5457,1.01,62.7,56.0,6.37,6.42,4.01,5,3,3,686.922026,1
40453,456,0.33,61.9,54.3,4.45,4.47,2.76,5,5,1,229.966844,1


In [35]:
diamonds_test["depth_normal"] = 0
for i, r in diamonds_test.iterrows():
    if 55 < r["depth"] < 68:
        diamonds_test.at[i, "depth_normal"] = 1
    else:
        diamonds_test.at[i, "depth_normal"] = 0
diamonds_test

,carat,depth,table,x,y,z,cut_encoded,clarity_encoded,color_encoded,volume,depth_normal
1,1.20,61.0,57.0,6.81,6.89,4.18,5,5,1,821.544750,1
2,1.57,62.2,61.0,7.38,7.32,4.57,4,3,3,1034.123131,1
3,0.90,63.8,54.0,6.09,6.13,3.90,3,3,5,609.861171,1
4,0.50,62.9,58.0,5.05,5.09,3.19,3,5,5,343.469717,1
5,1.01,61.3,61.0,6.38,6.34,3.90,4,4,5,660.789530,1
...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,61.9,56.0,5.35,5.32,3.30,5,3,6,393.430444,1
13481,0.71,62.2,55.0,5.71,5.73,3.56,5,4,2,487.898337,1
13482,0.70,61.6,55.0,5.75,5.71,3.53,5,5,5,485.475444,1
13483,0.70,58.8,57.0,5.85,5.89,3.45,3,2,5,497.942121,1


In [36]:
diamonds_train = diamonds_train.drop(["y", "z", "x"], axis=1)
diamonds_test = diamonds_test.drop(["y", "z", "x"], axis=1)

In [37]:
diamonds_train = diamonds_train.drop(["depth"], axis=1)
diamonds_test = diamonds_test.drop(["depth"], axis=1)

In [38]:
diamonds_train["table_normal"] = 0
for i, r in diamonds_train.iterrows():
    if 52 < r["table"] < 66:
        diamonds_train.at[i, "table_normal"] = 1
    else:
        diamonds_train.at[i, "table_normal"] = 0
diamonds_train

,price,carat,table,cut_encoded,clarity_encoded,color_encoded,volume,depth_normal,table_normal
0,4268,1.21,58.0,4,4,1,825.596831,1,1
1,505,0.32,57.0,3,4,3,219.474804,1,1
2,2686,0.71,55.0,1,5,4,475.163334,1,1
3,738,0.41,56.0,2,3,7,277.586100,1,1
4,4882,1.02,59.0,5,3,4,705.517829,1,1
...,...,...,...,...,...,...,...,...,...
40450,10070,1.34,57.0,5,5,4,927.520129,1,1
40451,12615,2.02,60.0,2,2,5,1358.328217,1,1
40452,5457,1.01,56.0,5,3,3,686.922026,1,1
40453,456,0.33,54.3,5,5,1,229.966844,1,1


In [39]:
diamonds_test["table_normal"] = 0
for i, r in diamonds_test.iterrows():
    if 55 < r["table"] < 68:
        diamonds_test.at[i, "table_normal"] = 1
    else:
        diamonds_test.at[i, "table_normal"] = 0
diamonds_test

,carat,table,cut_encoded,clarity_encoded,color_encoded,volume,depth_normal,table_normal
1,1.20,57.0,5,5,1,821.544750,1,1
2,1.57,61.0,4,3,3,1034.123131,1,1
3,0.90,54.0,3,3,5,609.861171,1,0
4,0.50,58.0,3,5,5,343.469717,1,1
5,1.01,61.0,4,4,5,660.789530,1,1
...,...,...,...,...,...,...,...,...
13480,0.57,56.0,5,3,6,393.430444,1,1
13481,0.71,55.0,5,4,2,487.898337,1,0
13482,0.70,55.0,5,5,5,485.475444,1,0
13483,0.70,57.0,3,2,5,497.942121,1,1


In [40]:
diamonds_train

,price,carat,table,cut_encoded,clarity_encoded,color_encoded,volume,depth_normal,table_normal
0,4268,1.21,58.0,4,4,1,825.596831,1,1
1,505,0.32,57.0,3,4,3,219.474804,1,1
2,2686,0.71,55.0,1,5,4,475.163334,1,1
3,738,0.41,56.0,2,3,7,277.586100,1,1
4,4882,1.02,59.0,5,3,4,705.517829,1,1
...,...,...,...,...,...,...,...,...,...
40450,10070,1.34,57.0,5,5,4,927.520129,1,1
40451,12615,2.02,60.0,2,2,5,1358.328217,1,1
40452,5457,1.01,56.0,5,3,3,686.922026,1,1
40453,456,0.33,54.3,5,5,1,229.966844,1,1


In [41]:
diamonds_train = diamonds_train.drop(["table"], axis=1)
diamonds_test = diamonds_test.drop(["table"], axis=1)

In [42]:
X = diamonds_train.drop(columns=['price'])
y = diamonds_train["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
regressor = RandomForestRegressor()

param_dist = {'n_estimators': randint(300, 325),  
              'max_depth': [None, 3, 5],  
              'min_samples_split': randint(2, 5), 
              'min_samples_leaf': randint(1, 3), 
              'max_features': ['sqrt']}  

random_search = RandomizedSearchCV(regressor, 
                                   param_distributions=param_dist, 
                                   n_iter=50, 
                                   cv=5, 
                                   verbose=3, 
                                   scoring='neg_root_mean_squared_error', 
                                   random_state=42, 
                                   n_jobs=-1)

random_search.fit(X_train, y_train)

print(random_search.best_estimator_)

y_pred = random_search.best_estimator_.predict(X_test)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 4/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=314;, score=-1089.705 total time=   4.6s
[CV 3/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=320;, score=-1107.500 total time=   4.4s
[CV 2/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=4, n_estimators=386;, score=-1073.969 total time=   5.4s
[CV 5/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=4, n_estimators=386;, score=-1089.390 total time=   5.8s
[CV 4/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=399;, score=-1087.954 total time=   5.6s
[CV 3/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=301;, score=-1091.892 total time=   4.7s
[CV 2/5] END max_depth=3, max_features=sqrt, min_samples_leaf=2, min_samples_split=3, n_estimators

[CV 1/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=314;, score=-1096.082 total time=   4.5s
[CV 5/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=314;, score=-1088.660 total time=   4.3s
[CV 4/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=320;, score=-1080.711 total time=   4.5s
[CV 3/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=4, n_estimators=386;, score=-1078.344 total time=   5.5s
[CV 2/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=399;, score=-1065.334 total time=   5.8s
[CV 1/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=301;, score=-1111.911 total time=   4.4s
[CV 5/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=301;, score=-1075.909 total time=   4.8s
[CV 5/5] END max_dep

[CV 3/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=314;, score=-1096.043 total time=   4.6s
[CV 2/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=320;, score=-1072.947 total time=   4.4s
[CV 1/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=4, n_estimators=386;, score=-1081.940 total time=   5.5s
[CV 1/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=399;, score=-1081.256 total time=   5.9s
[CV 5/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=399;, score=-1093.157 total time=   5.7s
[CV 4/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=301;, score=-1104.155 total time=   4.7s
[CV 3/5] END max_depth=3, max_features=sqrt, min_samples_leaf=2, min_samples_split=3, n_estimators=363;, score=-1601.731 total time=   3.7s
[CV 2/5] END max_dep

[CV 2/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=314;, score=-1079.743 total time=   4.6s
[CV 1/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=320;, score=-1081.391 total time=   4.4s
[CV 5/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=320;, score=-1058.034 total time=   4.5s
[CV 4/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=4, n_estimators=386;, score=-1082.940 total time=   5.6s
[CV 3/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=399;, score=-1087.193 total time=   5.8s
[CV 2/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=301;, score=-1081.989 total time=   4.5s
[CV 1/5] END max_depth=3, max_features=sqrt, min_samples_leaf=2, min_samples_split=3, n_estimators=363;, score=-1677.102 total time=   3.9s
[CV 4/5] END max_dep

RandomForestRegressor(max_features='sqrt', min_samples_split=3,
                      n_estimators=302)


In [48]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

/Users/SilviaMV/miniconda3/envs/sklearn-env/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


578.5952672598717

[CV 4/5] END max_depth=3, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=305;, score=-1577.847 total time=   3.0s
[CV 3/5] END max_depth=3, max_features=sqrt, min_samples_leaf=2, min_samples_split=4, n_estimators=327;, score=-1609.030 total time=   3.2s
[CV 2/5] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=3, n_estimators=332;, score=-573.565 total time=  17.9s
[CV 1/5] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=312;, score=-567.741 total time=  14.4s
[CV 4/5] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=312;, score=-553.542 total time=  15.5s
[CV 3/5] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=305;, score=-526.062 total time=  12.0s
[CV 3/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=314;, score=-1108.471 total time=   4.4s
[CV 5/5] END

[CV 4/5] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=3, n_estimators=336;, score=-571.351 total time=  14.1s
[CV 3/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=314;, score=-1065.472 total time=   4.5s
[CV 1/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=331;, score=-1074.369 total time=   4.8s
[CV 5/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=331;, score=-1096.651 total time=   4.6s
[CV 4/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=321;, score=-1072.112 total time=   4.6s
[CV 3/5] END max_depth=3, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=305;, score=-1607.158 total time=   3.1s
[CV 2/5] END max_depth=3, max_features=sqrt, min_samples_leaf=2, min_samples_split=4, n_estimators=327;, score=-1585.814 total time=   3.2s
[CV 1/5] END max_d

[CV 5/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=314;, score=-1086.151 total time=   4.8s
[CV 3/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=3, n_estimators=312;, score=-1066.353 total time=   5.0s
[CV 2/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=348;, score=-1083.349 total time=   5.3s
[CV 1/5] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=3, n_estimators=336;, score=-579.724 total time=  14.0s
[CV 5/5] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=3, n_estimators=336;, score=-604.523 total time=  13.8s
[CV 4/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=331;, score=-1064.933 total time=   4.6s
[CV 3/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=321;, score=-1083.326 total time=   4.6s
[CV 2/5] END max

[CV 5/5] END max_depth=3, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=305;, score=-1613.343 total time=   3.0s
[CV 4/5] END max_depth=3, max_features=sqrt, min_samples_leaf=2, min_samples_split=4, n_estimators=327;, score=-1583.774 total time=   3.2s
[CV 3/5] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=3, n_estimators=332;, score=-526.367 total time=  17.5s
[CV 2/5] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=312;, score=-571.391 total time=  14.3s
[CV 5/5] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=312;, score=-589.038 total time=  15.7s
[CV 4/5] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=305;, score=-555.602 total time=  11.8s
[CV 4/5] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=314;, score=-1073.200 total time=   4.5s
[CV 3/5] END

In [ ]:
y_prediction = random_search.predict(diamonds_test)

In [ ]:
solution = []
for i, value in enumerate(y_prediction):
    solution.append((i,value))

In [ ]:
submission_file = pd.DataFrame(solution)

In [ ]:
submission_file

In [ ]:
submission_file.columns = ["id", "price"]

In [ ]:
submission_file.reset_index(drop=True)